In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML
import scipy.stats as stats
from scipy.stats import iqr

def display_figure(fig, fig_id, caption, w=0.6, fs=12):
    import io
    import base64
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    pic_IObytes = io.BytesIO()
    
    # set fontsize for title and labels        
    for ax in fig.axes:
        text_items = [ax.title, ax.xaxis.label, ax.yaxis.label]
        if ax.get_legend() is not None:
            text_items = text_items + ax.get_legend().get_texts()
        for item in (text_items + ax.get_xticklabels() + ax.get_yticklabels() ):
            item.set_fontsize(fs)
            
    plt.savefig(pic_IObytes,  format='png', bbox_inches='tight')
    pic_IObytes.seek(0)
    pic_hash = base64.b64encode(pic_IObytes.read())
    img = f'<img margin="auto" width="{w*100}%" src="data:image/png;base64, {str(pic_hash)[2:-1]}" />'
    caption = f'<figcaption style="text-align: center; font-style: italic;">{caption}</figcaption>'
    display(HTML(f'<br><fig width="100%" id={fig_id}><center>{img}</center> {caption}</fig><br>'))
    plt.close(fig)

In [2]:
ANS_df = pd.read_csv("ANS_responses.csv")
Math_df = pd.read_csv("Math_responses.csv")
Memory_df = pd.read_csv("Memory_responses.csv")
SRT_df = pd.read_csv("SRT_responses.csv")
SRT_df = SRT_df.rename(columns={"total_score":"score"})
SRT_df = SRT_df.rename(columns={"user_id":"id"})
SRT_df = SRT_df.set_index("id")
SRT_df = SRT_df.loc[:,["gender","score"]]
Memory_df = Memory_df.rename(columns={"points":"score"})
Memory_df = Memory_df.rename(columns={"username":"id"})
Memory_df = Memory_df.set_index("id")
Memory_df = Memory_df.loc[:,["gender","score"]]
Math_df = Math_df.set_index("id")
Math_df = Math_df.loc[:,["gender","score"]]
ANS_df = ANS_df.set_index("id")
ANS_df = ANS_df.loc[:,["gender","score"]]
ANS_df = ANS_df.drop(["jy", "ydn"], axis = "rows")
ANS_df.to_csv("ANS_responses_processed.csv")
Math_df.to_csv("Math_responses_processed.csv")
Memory_df.to_csv("Memory_responses_processed.csv")
SRT_df.to_csv("SRT_responses_processed.csv")

## Hypotheses B

This report also investigates the relationship between the test scores (ANS, Math, Memory, and SRT) and the gender of the participants. The participants were divided into three gender categories: male, female, and others. Since only two participants identified themselves as ‘other’, their data were removed and the report will be focussing on comparing the score obtained between Male and Female participants.

Boxplots can be drawn to identify any outliers that may affect the process of analysing the data.

In [8]:
fig, axs = plt.subplots(1,4, figsize=(12,4))
ax1 = ANS_df.boxplot(column="score", by="gender", ax=axs[0])
ax1.set_title("ANS")
ax1.set_ylabel("score", fontsize=8)
ax2 = Math_df.boxplot(column="score", by="gender", ax=axs[1])
ax2.set_title("Math")
ax2.set_ylabel("score", fontsize=8)
ax3 = Memory_df.boxplot(column="score", by="gender", ax=axs[2])
ax3.set_title("Memory")
ax3.set_ylabel("score", fontsize=8)
ax4 = SRT_df.boxplot(column="score", by="gender", ax=axs[3])
ax4.set_title("SRT")
ax4.set_ylabel("score", fontsize=8)
plt.tight_layout()


caption = "Figure 1:Boxplots of scores for each tests(ANS, Math, Memory, SRT) seperated by gender"
display_figure(fig, "fig", caption, 0.7, 10)


Figure 1 shows that there are outliers from the score obtained in ANS test and Math test. These outliers need to be removed as it can lead to an inaccurate interpretation of the data. To remove the outlier, the interquartile range, upper bound and lower bound of the score for ANS test and Math test needs to be calculated. The code below demonstrate how to calculate these data and use it to remove the outliers.

In [4]:
def remove_outliers(dataframe):
    Q1 = dataframe['score'].quantile(0.25)
    Q3 = dataframe['score'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_cleaned = dataframe[(dataframe['score'] >= lower_bound) & (dataframe['score'] <= upper_bound)]
    return df_cleaned

ANS_df = ANS_df.groupby('gender').apply(remove_outliers).reset_index(drop=True)
Math_df = Math_df.groupby('gender').apply(remove_outliers).reset_index(drop=True)
ANS_df.to_csv("ANS_responses_processed.csv")
Math_df.to_csv("Math_responses_processed.csv")


The boxplots indicate that male participants outperformed female participants in all but the ANS test. This suggests that gender may play an important role in determining participants' test scores. However, observing these boxplots cannot lead to a conclusion because they overlap significantly. Thus, a t-test must be performed to ensure that the observed differences are statistically significant.

For a t-test to be done, several data need to be calculated which includes:
- `mean_score_male` the mean score obtained by the Male participants for each tests.
- `sd_score_male` the standard deviation of the scores obtained by the Male Participants.
- `mean_score_female` the mean score obtained by the Male participants for each tests.
- `sd_score_female` the standard deviation of the scores obtained by the Female Participants.
- `score_diff` the differences of the score mean between the Male and Female Participants

The code and table below demonstrate how to calculate these data and the p-value of the t-test for each corresponding test:

In [5]:
def calculations_data(test ,processed_filename):
    test_df = pd.read_csv(processed_filename)
    results ={}
    results['mean_score_male'] = test_df.query('gender== "male" | gender=="Male"').score.mean()
    results['sd_score_male'] = test_df.query('gender=="male" | gender=="Male"').score.std()
    results['mean_score_female'] = test_df.query('gender== "female" | gender=="Female"').score.mean()
    results['sd_score_frmale'] = test_df.query('gender=="female" | gender=="Female"').score.std()
    results['score_diff'] = results['mean_score_male'] - results['mean_score_female']
    sig_test = stats.ttest_ind(test_df.query('gender== "male" | gender=="Male"').score, test_df.query('gender== "female" | gender=="Female"').score)
    results['p-value'] = sig_test.pvalue
    return results


    
    

In [9]:
subject_ids = ["ANS", "Math", "Memory", "SRT"]

data_set = []
for subject_id in subject_ids:
    processed_filename = subject_id+"_responses_processed.csv"
    test = subject_id
    result = calculations_data(test, processed_filename)
    result['id'] = subject_id
    data_set.append(result)

calc_df = pd.DataFrame(data_set)


caption = "Figure 2: Table for the mean, standard deviation, difference in score, and p-value for each test by gender "
caption = f'<figcaption style="text-align: center; font-style: italic;">{caption}</figcaption>'
display(calc_df.set_index("id"))
display(HTML(caption))

,mean_score_male,sd_score_male,mean_score_female,sd_score_frmale,score_diff,p-value
id,,,,,,
ANS,51.941176,2.384077,51.777778,5.219371,0.163399,0.906852
Math,13.080000,1.320353,12.000000,1.865010,1.080000,0.023177
Memory,10.642857,3.346798,10.272727,3.268636,0.370130,0.696678
SRT,2.833333,1.947849,2.666667,1.940285,0.166667,0.798581


In [10]:
fig = plt.figure(figsize=(8,8))

ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(ANS_df['gender'], ANS_df['score'], '.')
ax1.set_xlim(-1,2)
ANS_m_mean = ANS_df.query('gender== "male" | gender=="Male"').score.mean()
ANS_m_std = stats.sem(ANS_df.query('gender== "male" | gender=="Male"').score)
ANS_f_mean = ANS_df.query('gender== "female" | gender=="Female"').score.mean()
ANS_f_std = stats.sem(ANS_df.query('gender== "female" | gender=="Female"').score)
ax1.errorbar('Male', ANS_m_mean, yerr =ANS_m_std, marker='_', color ='black')
ax1.errorbar('Female', ANS_f_mean, yerr =ANS_f_std, marker='_', color ='black')
result = stats.ttest_ind(ANS_df.query('gender== "male" | gender=="Male"').score, ANS_df.query('gender== "female" | gender=="Female"').score)
ax1.set_title(f"ANS(p-value:{result.pvalue:.3g})")

ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(Math_df['gender'], Math_df['score'], '.')
ax2.set_xlim(-1,2)
Math_m_mean = Math_df.query('gender== "male" | gender=="Male"').score.mean()
Math_m_std = stats.sem(Math_df.query('gender== "male" | gender=="Male"').score)
Math_f_mean = Math_df.query('gender== "female" | gender=="Female"').score.mean()
Math_f_std = stats.sem(Math_df.query('gender== "female" | gender=="Female"').score)
ax2.errorbar('male', Math_m_mean, yerr =Math_m_std, marker='_', color ='red')
ax2.errorbar('female', Math_f_mean, yerr =Math_f_std, marker='_', color ='red')
result = stats.ttest_ind(Math_df.query('gender== "male" | gender=="Male"').score, Math_df.query('gender== "female" | gender=="Female"').score)
ax2.set_title(f"Math(p-value:{result.pvalue:.3g})")

ax3 = fig.add_subplot(2, 2, 3)
ax3.plot(Memory_df['gender'], Memory_df['score'], '.')
ax3.set_xlim(-1,2)
Memory_m_mean = Memory_df.query('gender== "male" | gender=="Male"').score.mean()
Memory_m_std = stats.sem(Memory_df.query('gender== "male" | gender=="Male"').score)
Memory_f_mean = Memory_df.query('gender== "female" | gender=="Female"').score.mean()
Memory_f_std = stats.sem(Memory_df.query('gender== "female" | gender=="Female"').score)
ax3.errorbar('male', Memory_m_mean, yerr =Memory_m_std, marker='_', color ='red')
ax3.errorbar('female', Memory_f_mean, yerr =Memory_f_std, marker='_', color ='red')
result = stats.ttest_ind(Memory_df.query('gender== "male" | gender=="Male"').score, Memory_df.query('gender== "female" | gender=="Female"').score)
ax3.set_title(f"Memory(p-value:{result.pvalue:.3g})")

ax4 = fig.add_subplot(2, 2, 4)
ax4.plot(SRT_df['gender'], SRT_df['score'], '.')
ax4.set_xlim(-1,2)
SRT_m_mean = SRT_df.query('gender== "male" | gender=="Male"').score.mean()
SRT_m_std = stats.sem(SRT_df.query('gender== "male" | gender=="Male"').score)
SRT_f_mean = SRT_df.query('gender== "female" | gender=="Female"').score.mean()
SRT_f_std = stats.sem(SRT_df.query('gender== "female" | gender=="Female"').score)
ax4.errorbar('Male', SRT_m_mean, yerr =SRT_m_std, marker='_', color ='red')
ax4.errorbar('Female', SRT_f_mean, yerr =SRT_f_std, marker='_', color ='red')
result = stats.ttest_ind(SRT_df.query('gender== "male" | gender=="Male"').score, SRT_df.query('gender== "female" | gender=="Female"').score)
ax4.set_title(f"SRT(p-value:{result.pvalue:.3g})")

caption = "Figure 3: Comparison of test scores(ANS, Math, Memory and Spatial Recognition tests) by gender"
display_figure(fig, "fig", caption, 0.5, 10)

Based on Figure 3, the result of the comparisons can be observed. At a 5% significance level, it can be observed that there are no significant differences between the score obtained by these two groups in the ANS, Memory and Spatial Recognition test. However, the difference in score that are being observed in the Math is statistically significant. This suggest that gender may effect the participants' outcome when taking Math test.